## RoniAnalytics Assessment Task

Fetched the data using psycopg2 by connecting to the server and downloaded it in csv format

In [ ]:
import psycopg2
import io
import csv
import requests
from psycopg2 import sql

host = 'ethreum-rds.cluster-crwz9eu9x7fl.ap-south-1.rds.amazonaws.com'
database = 'dev'
username = 'price_data_read_user'
password = 'B9c1UNF62jjZ'

conn = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
)

# total rows = 65252874
query = "SELECT * FROM {table} LIMIT 7000 OFFSET 0;"

cursor=conn.cursor()
cursor.execute(
    sql.SQL(query)
    .format(table=sql.Identifier('btcusdt-ochlv')))
col_names =[desc[0] for desc in cursor.description]

with open('dataminute.csv',mode='w',newline='') as file:
    writer = csv.writer(file)
    writer.writerow(col_names)

with open('dataminute.csv',mode='a',newline='') as file:
    writer = csv.writer(file)
    for row in cursor.fetchall():
        writer.writerow(row)
cursor.close()
conn.close()

## Implemented SSL Hybrid Indicator using Plotly


In [ ]:
import pandas as pd
import numpy as np
import plotly.offline as pyo
from ta.trend import MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
import talib
import ta
import plotly.graph_objs as go
import plotly.subplots as sp

All these libraries help in providing useful functions inorder to implement the indicators, plot the charts and reading the dataset

In [ ]:
# Load data from CSV file
df = pd.read_csv('./datamin.csv')

The line loads the data from CSV file named 'datamin.csv' using pandas and stores it in a pandas Dataframe named 'df'.

In [ ]:
df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

This line adds a new column called 'RSI' to the DataFrame df containing the Relative Strength Index (RSI) indicator computed using the ta.momentum.RSIIndicator function with a window size of 14.

In [ ]:
df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()

In [ ]:
period = 10
p2=5
dev=2

we are calculating the upper and lower SuperTrend values using the High, Low, and Close prices of the DataFrame df. The ssl_u and ssl_d values are calculated using the following formulas:

Upper SuperTrend (ssl_u) = (High + Low) / 2 + dev * sqrt((High - Low)^ 2 / period)

Lower SuperTrend (ssl_d) = (High + Low) / 2 - dev * sqrt((High - Low)^ 2 / period)

Here, np.sqrt () that calculates the squareroot of the argument passed to it.


In [ ]:
ssl_u = (df['High'] + df['Low']) / 2 + dev * np.sqrt((df['High'] - df['Low']) ** 2 / period)
ssl_d = (df['High'] + df['Low']) / 2 - dev * np.sqrt((df['High'] - df['Low']) ** 2 / period)

we are calculating the Hilbert Transform - Instantaneous Trendline for both the upper and lower SuperTrend values using the talib.HT_TRENDLINE() function from the ta-lib library. The ssl_up and ssl_down values are stored as the output of the talib.HT_TRENDLINE() function.

In [ ]:
ssl_up = talib.HT_TRENDLINE(ssl_u)
ssl_down = talib.HT_TRENDLINE(ssl_d)

we are calculating the Exponential Moving Average (EMA) of the Close prices of the DataFrame df. The talib.EMA() function from the ta-lib library is used for this calculation, and we pass the timeperiod argument as period (which we assigned the value 10 to earlier).

In [ ]:
ema = talib.EMA(df['Close'], timeperiod=period)

In these lines of code, we are calculating the SuperTrend Fast and SuperTrend Slow values.
First, we calculate the difference between ssl_up and ssl_down, which gives us the oscillator value. Next, we calculate the EMA of the oscillator value using the talib.EMA() function and store it in ssl_first. Finally, we calculate the EMA of ssl_fast to get the SuperTrend Slow value and store it in ssl_slow.

In [ ]:
oscillator = ssl_up - ssl_down
ssl_fast = talib.EMA(oscillator, timeperiod=period)
ssl_slow = talib.EMA(ssl_fast, timeperiod=period)

we are creating a new column in df called ssl_hybrid, which is populated with the values 1 or 0 based on a condition. If ssl_fast is greater than ssl_slow, then the corresponding value in ssl_hybrid is set to 1; otherwise, it is set to 0. The np.where() function from the NumPy library is used to perform this operation.

In [ ]:
ssl_hybrid = np.where(ssl_fast > ssl_slow, 1, 0)

Here we set bollinger_period to 20 which means that the moving average will be calculated based on the last 20 periods, bollinger deviation is set to 2 which is the number of standard deviations from the moving average that will be used to calculate the Upper and lower bands

The BollingerBands class from ta library is used to calculate the upper and lower bands based on the given parameters. The BollingerBands class takes in the closing prices stock as input, along with the window and window_dev parameters

Finally, the upper and lower bands are added to the original dataframe using the bollinger_hband() and bollinger_lband() methods from the BollingerBands object, respectively. The resulting upper and lower bands are added to the dataframe as new columns named bb_hband and bb_lband respectively.

In [ ]:
# Set the Bollinger Bands parameters
bollinger_period = 20
bollinger_deviation = 2

# Calculate the Bollinger Bands using the BollingerBands class from ta library
bb = BollingerBands(df['Close'], window=bollinger_period, window_dev=bollinger_deviation)

# Add the upper and lower bands to the DataFrame
df['bb_hband']= bb.bollinger_hband()
df['bb_lband']= bb.bollinger_lband()

The next part calculates the Moving Average Convergence Divergence (MACD) and its corresponding signal line and histogram.

Calculate the 26-period exponential moving average (EMA) and the 12-period EMA of the closing price using the pandas ewm() function. The span parameter in the function specifies the number of time periods to use for calculating the EMA.

In [ ]:
df['26 EMA'] = df['Close'].ewm(span=26).mean()
df['12 EMA'] = df['Close'].ewm(span=12).mean()

Calculate the MACD line by subtracting the 26-period EMA from the 12-period EMA.

In [ ]:
df['MACD'] = df['12 EMA'] - df['26 EMA']

Calculate the signal line, which is a 9-period EMA of the MACD line.

In [ ]:
df['Signal Line'] = df['MACD'].ewm(span=9).mean()

Calculate the MACD histogram, which is the difference between the MACD line and the signal line.

In [ ]:
df['MACD Histogram'] = df['MACD'] - df['Signal Line']

The resulting values for the MACD, signal line, and MACD histogram are added as new columns to the DataFrame df.

We first set the period for the Relative Strength Index (RSI) to 14.
Then, we calculate the RSI values for the Close column of the df DataFrame using the RSIIndicator class from the ta library.
We add the RSI values to a new column named rsi in the df DataFrame.

In [ ]:
rsi_period = 14
rsi= RSIIndicator(df['Close'], rsi_period)
df['rsi']=rsi.rsi()

We add column named sign.
We use the loc function to filter the rows of the df DataFrame where the RSI value is less than 30, the close price is below the lower Bollinger Band, and the MACD value is greater than the Signal Line. We set the value of the sign column in these rows to 1.
We use the loc function again to filter the rows of the df DataFrame where the RSI value is less than 30, the close price is below the lower Bollinger Band, and the MACD value is less than the Signal Line. We set the value of the sign column in these rows to -1.


In [ ]:
df.loc[(df['rsi'] < 30) & (df['Close'] < df['bb_lband']) & (df['MACD'] > df['Signal Line'])  ,'sign']= 1
df.loc[(df['rsi'] < 30) & (df['Close'] < df['bb_lband']) & (df['MACD'] < df['Signal Line']) ,'sign']= -1

Creating 2 new dataframes named buy_signals and sell_signals. buy_signals contains the rows from df where the value of the sign column is equal to 1. sell_signals contains the rows from df where the value of the sign column is equal to -1.

In [ ]:
buy_signals = df[df['sign'] == 1]
sell_signals = df[df['sign'] == -1]

create candlestick chart with SSL up and down traces, and buy and sell arrow signals. This is done by creating four traces. go.Candlestick creates a candlestick chart, while go.Scatter creates lines for the SSL up and down traces. We pass the necessary data to the traces using the x and y parameters.

In [ ]:
candle = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Candlestick')
ssl_up_trace = go.Scatter(x=df.index, y=ssl_up, mode='lines', line=dict(color='blue'), name='SSL Up')
ssl_down_trace = go.Scatter(x=df.index, y=ssl_down, mode='lines', line=dict(color='red'), name='SSL Down')

In a similar way we create bb_upper and bb_lower plots

In [ ]:
bb_upper = go.Scatter(x=df.index, y=df['bb_hband'], name='BB Upper', line=dict(color='#ccc', width=1, dash='dot'))
bb_lower = go.Scatter(x=df.index, y=df['bb_lband'], name='BB Lower', line=dict(color='#ccc', width=1, dash='dot'))

we then add all of these traces to a list called data and append the list with Bollinger Bands Upper and Lower traces 

In [ ]:
data = [candle, ssl_up_trace, ssl_down_trace]

data.append(bb_lower)
data.append(bb_upper)

This code creates a bar chart object using plotly.graph_objs for the volume with the following arguments:
X: the x-axis values, which are the index of the dataframe (date and time)
y: the values of the volume
name: the name of the volume, which is 'volume'

In [ ]:
volume = go.Bar(x=df['datetime'], y=df['Volume'], name='Volume')

a subplot object is created with 3 rows and 1 column, where 'shared_xaxes=True' specifies that all subplots share the same x-axis, and 'vertical_spacing=0.05' adds a small vertical spacing between the subplots.  

In [ ]:
fig = sp.make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

the candlestick chart, SSL up and down traces, and Bollinger Bands are added to the first row of the subplot

In [ ]:
fig.add_traces(data, rows=1, cols=1)

 the volume bars are added to the third row of the subplot.

In [ ]:
fig.add_trace(volume, row=3, col=1)

a scatter plot with yellow upward triangles is added to mark the buy signals.

In [ ]:
fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', marker=dict(symbol='triangle-up', size=10, color='yellow'), name='Buy'))

a scatter plot with black downward triangles is added to mark the sell signals.

In [ ]:
fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', marker=dict(symbol='triangle-down', size=10, color='black'), name='Sell'))

a line plot of the MACD values is added to the second row of the subplot.

In [ ]:
fig.add_trace(go.Scatter(x=df['datetime'], y=df['MACD'], mode='lines', name='MACD'), row=2, col=1)

a line plot of the signal line values is added to the second row of the subplot.

In [ ]:
fig.add_trace(go.Scatter(x=df['datetime'], y=df['Signal Line'], mode='lines', name='Signal Line'), row=2, col=1)

a bar chart of the MACD histogram is added to the second row of the subplot.

In [ ]:
fig.add_trace(go.Bar(x=df['datetime'], y=df['MACD Histogram'], name='MACD Histogram'), row=2, col=1)

the layout of the subplot is updated by setting xaxis_rangeslider_visible=False to remove the x-axis range slider and height=600 to set the height of the subplot to 600 pixels.

In [ ]:
fig.update_layout(xaxis_rangeslider_visible=False, height=600)

Below line saves the plot created using Plotly as an HTML file names SSL-Hybrid.html 

In [ ]:
fig.write_html("./SSL-Hybrid.html")

The show() function displays the plot

In [ ]:
fig.show()

# QQE-Mod Indicator Implementation

The period of the exponential moving averages (EMAs) and RSI is set to 14 and then the EMA of bitcoin price data is calculated and stored in a new column named "EMA" using ewm function

In [ ]:
period=14
df['EMA'] = df['Close'].ewm(span=period).mean()

And then the RSI of Bitcoin price data is calculated and stored in a new column named "RSI". The delta between each day's closing price and the previous day's closing price is calculated using the diff function. The gains and losses are then calculated using the where function, and the rolling averages are calculated using the rolling function. 

In [ ]:
delta = df['Close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(period).mean()
avg_loss = loss.rolling(period).mean()
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

The MACD and signal line are calculated and stored in new columns named "MACD" and "Signal", respectively, using the ewm function.

In [ ]:
exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = exp1 - exp2
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

The smoothed RSI (sRSI) is calculated by applying an EMA to the RSI values.

In [ ]:
df['sRSI'] = df['RSI'].ewm(span=period).mean()

The QQE indicator is calculated by applying an EMA to the difference between the MACD and Signal values, and multiplying it by the sRSI.

In [ ]:
df['MACD_diff'] = df['MACD'] - df['Signal']
df['QQE'] = df['sRSI'] * df['MACD_diff'].ewm(span=period).mean()

The QQE Mod indicator is calculated by applying an EMA to the QQE values.

In [ ]:
df['QQE_Mod'] = df['QQE'].ewm(span=period).mean()

Creating a new column in the DataFrame called "Trend", which contains the trend direction for each data point. It iterates over the QQE Mod indicator values in the DataFrame and compares each value to the previous value to determine the trend direction. If the current value is greater than the previous value and greater than zero, it is considered bullish and appended to the trend list as "Bullish". If the current value is less than the previous value and less than zero, it is considered bearish and appended to the trend list as "Bearish". Otherwise, it is considered consolidating and appended to the trend list as "Consolidating". The first value in the trend list is inserted as "Consolidating" to ensure that the length of the trend list matches the length of the DataFrame.

In [ ]:
trend = []
for i in range(1, len(df)):
    if df['QQE_Mod'][i] > df['QQE_Mod'][i-1] and df['QQE_Mod'][i] > 0:
        trend.append('Bullish')
    elif df['QQE_Mod'][i] < df['QQE_Mod'][i-1] and df['QQE_Mod'][i] < 0:
        trend.append('Bearish')
    else:
        trend.append('Consolidating')
trend.insert(0, 'Consolidating')
df['Trend'] = trend

This code filters out any data points where the QQE Mod indicator is equal to zero

df = df[df['QQE_Mod'] != 0]

creating a color-coded bar chart using Plotly to visualize the QQE Mod indicator for Bitcoin price. It creates a list of colors based on the trend direction and QQE Mod values in the DataFrame. If the trend is bearish and the QQE Mod value is less than zero, the color is set to red. If the trend is consolidating, the color is set to gray. If the trend is bullish or the QQE Mod value is greater than or equal to zero, the color is set to green. It then creates a Figure object with a bar chart using the QQE Mod values and the color list. It sets the marker line width to zero to remove the border around the bars. It updates the layout with a title and axis labels. Finally, it writes the chart to an HTML file and displays the chart

In [ ]:
colors = ['red' if trend == 'Bearish' and QQE_Mod < 0 else 'gray' if trend == 'Consolidating' else 'green' for trend, QQE_Mod in zip(df['Trend'], df['QQE_Mod'])]
fig = go.Figure(data=[go.Bar(x=df.index, y=df['QQE_Mod'], marker={'color': colors}, hovertemplate='<b>QQE Mod:</b> %{y:.2f}<extra></extra>')])
fig.update_traces(selector=dict(type='bar'), marker_line_width=0)
fig.update_layout(title='QQE Mod Indicator for Bitcoin Price', xaxis_title='Date', yaxis_title='QQE Mod', yaxis_tickformat='.2f')
fig.write_html("./QQE-Mod.html")
fig.show()

## Trading strategy for SSL Hybrid Indicator 

SSL Hybrid indicator is calculated using the high and low prices of the instrument over a certain period. In this case, the period is set to 10 and the deviation factor is set to 2. The SSL Hybrid indicator is made up of an oscillator made from the SSL indicator and two Exponential Moving Averages (EMA). The oscillator is found by taking the difference between the SSL up trendline and the SSL down trendline. If the fast EMA is higher than the slow EMA, it sends a buy signal, and if the fast EMA is lower than the slow EMA, it sends a sell signal.

Among the other indicators used in the implementation are:
Bollinger Bands: These are made by taking the closing price of the instrument and figuring out where the price is too high or too low.
MACD, which stands for Moving Average Convergence Divergence, is a momentum indicator that shows the relationship between two moving averages of the price of an instrument.
RSI (Relative Strength Index): This is a momentum indicator that looks at how much prices have changed recently to see if the instrument is overbought or oversold.

## Trading strategy implementation for QQE-Mod Indicator

Used the closing price for a timeperiod = 14 to figure out the exponential moving average (EMA) of the bitcoin price and then calculated the relative strength index (RSI)) of the bitcoin price using the closing price for the considered period. The RSI is a momentum oscillator that measures how quickly prices move and how they change and then by using the closing price for two different periods to figure out the moving average convergence divergence (MACD) and the signal line and then calculated the smoothed RSI (sRSI) by applying an EMA to the RSI values. And then applied an exponential moving average to the difference between MACD and signal values and then multiplied that number by the sRSI this will gives us the QQE-Mod Indicator. Calculated the QQE Mod indicator by applying an EMA to the QQE values and compared the current and previous QQE Mod values to figure out the direction and strength of the trend. If the current value is higher than the last value and the difference is positive is called a bullish trend. If the current value is less than the previous value and is negative then the trend is considered as bearish. If the current value is neither greater nor less than the previous value and is not equal to 0, the trend is considered as consolidating. And then filtered out the data points where QQQE Mod indicator is zero and made color-coded bar chart to see the QQE Mod indicator and the direction of the trend using Plotly 